In [64]:
import spacy
import pandas as pd
import re
import jsonlines
from utils.prepare_rules import create_rules 
from utils.prepare_data import create_prepare_file
from tqdm import tqdm
import os.path
from spacy.language import Language

In [65]:
DATA='data/pledges - pledges.csv'
BRANDS='data/brands.jsonl'
RULES='data/rules.jsonl'
DATA_PREPARE='data/pledges_prepare.csv'

BRAND_PATH = 'data/brands/'
RULES_PATH = 'data/rules/'

#### Загружаем данные для парсинга и данные по брендам

In [66]:
data = pd.read_csv(DATA, delimiter='|')

In [67]:
brands = list(jsonlines.open(BRANDS))
list_brands = [x['id'] for x in brands]

### Генерируем правила для моделей и марок и сохраняем их в файлы

In [68]:
rules = create_rules(brands)

# save in file
with jsonlines.open(RULES, mode='w') as writer:
    writer.write_all(rules)
    
for brand in list_brands:
    rules_model = create_rules(brands)
    try:
        models = jsonlines.open(BRAND_PATH + brand +'.jsonl')
        rules_model = create_rules(models, label='MODEL')
        with jsonlines.open(RULES_PATH + brand + '_rules.jsonl', mode='w') as writer:
            writer.write_all(rules_model)
    except:
        print('Для бренда '+ brand+ ' не найден файл с моделями')

Для бренда ACURA не найден файл с моделями
Для бренда ADR GRUP не найден файл с моделями
Для бренда ALLOY не найден файл с моделями
Для бренда AMUR не найден файл с моделями
Для бренда ASIA не найден файл с моделями
Для бренда ATLETIC не найден файл с моделями
Для бренда AVTOMASH не найден файл с моделями
Для бренда AYATS не найден файл с моделями
Для бренда BENTLEY не найден файл с моделями
Для бренда BLUMHARDT не найден файл с моделями
Для бренда BRILLIANCE не найден файл с моделями
Для бренда BSS-METACO не найден файл с моделями
Для бренда BUNGE не найден файл с моделями
Для бренда CAMC не найден файл с моделями
Для бренда CAT не найден файл с моделями
Для бренда CHAIKA-SERVIS не найден файл с моделями
Для бренда CHANGAN не найден файл с моделями
Для бренда CHANGFENG не найден файл с моделями
Для бренда CHEREAU не найден файл с моделями
Для бренда CHERRY не найден файл с моделями
Для бренда DACIA не найден файл с моделями
Для бренда DNESTR не найден файл с моделями
Для бренда DOOSAN

### Попытаемся отбить пробелы
Возможно это не слишком хорошая идея...

In [69]:
list_models = []
for brand in list_brands:
    try:
        reader = jsonlines.open('data/brands/'+ brand +'.jsonl')
        list_models = list_models + list(reader)
    except:
        pass

In [70]:
str_with_space = create_prepare_file(data=data['vehicleproperty_description_short'].values, brands=brands+list_models)

data['with_space'] = str_with_space
data.to_csv(DATA_PREPARE, index=False)

In [71]:
data = pd.read_csv(DATA_PREPARE, delimiter=',')

### NER

In [72]:
rules = list(jsonlines.open(RULES))
for brand in list_brands:
    if os.path.exists(RULES_PATH + brand + '_rules.jsonl'):
            file = jsonlines.open(RULES_PATH + brand + '_rules.jsonl')
            rules = rules + (list(file))

In [73]:
nlp = spacy.load("ru_core_news_sm",exclude=['tok2vec', 'morphologizer', 'parser', 'senter', 'attribute_ruler', 'lemmatizer'])

config = { "overwrite_ents": True }
ruler = nlp.add_pipe("entity_ruler", before="ner", config=config) #.from_disk(RULES_PATH +"all_rules.jsonl", )
ruler.add_patterns(rules)

# @Language.component("expand_model")
# def expand_model(doc):
#     new_ents = []
#     for ent in doc.ents:
#         if ent.label_ == "BRAND" and ent.ent_id_ == 'lada':
#                 print([x for x in doc])
# #             print(doc)
# #             prev_token = doc[ent.start - 1]
# #             if prev_token.text in ("Dr", "Dr.", "Mr", "Mr.", "Ms", "Ms."):
# #                 new_ent = Span(doc, ent.start - 1, ent.end, label=ent.label)
# #                 new_ents.append(new_ent)
#         else:
#             new_ents.append(ent)
# #     doc.ents = new_ents
#     return doc

# nlp.add_pipe("expand_model", after="ner")

brand = []
model = []
for article in tqdm(data['with_space']):
    article = str(article)
    doc = nlp(article)
    list_b = []
    list_m = []
    for ent in doc.ents:
        if ent.label_ == 'BRAND': list_b.append(ent.ent_id_)
        if ent.label_ == 'MODEL': list_m.append(ent.ent_id_)
    brand.append(None if len(list_b) == 0 else list_b[0])
    model.append(None if len(list_m) == 0 else list_m[0])

data = data.assign(Brand_NER=brand)
data = data.assign(Model_NER=model)


100%|███████████████████| 10000/10000 [00:53<00:00, 186.17it/s]


In [74]:
data['Brand_NER'].value_counts()

lada             764
mercedes-benz    605
toyota           455
kia              431
hyundai          386
                ... 
saturn             1
asia               1
pontiac            1
chereau            1
peterbilt          1
Name: Brand_NER, Length: 125, dtype: int64

In [75]:
data['Brand_NER'].isnull().value_counts()

False    8358
True     1642
Name: Brand_NER, dtype: int64

In [76]:
data['Model_NER'].isnull().value_counts()

False    6070
True     3930
Name: Model_NER, dtype: int64

In [79]:
data[data["Brand_NER"].isnull()][data["vehicleproperty_description_short"] != 'Автомобиль'][1000:1050]

/var/folders/2t/mb7xtxlx289fhmxl983vql2c0000gp/T/ipykernel_73591/4124886449.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data[data["Brand_NER"].isnull()][data["vehicleproperty_description_short"] != 'Автомобиль'][1000:1050]


,id,notificationreferencenumber,vehicleproperty_vin,vehicleproperty_description_short,nBody,nChas,BrandModel,BrandModel2,BrandModel3,Brand,Brand2,with_space,Brand_NER,Model_NER
7438,29523,2014-000-006726-104,X90962264C0005770,Полуприцеп-цистерна 96226-0000010-04год изгото...,NaN,NaN,NaN,NaN,NaN,GAZ,NaN,Полуприцеп-цистерна 96226-0000010-04год изгото...,None,None
7439,29524,2014-000-006726-104,X90962264C0005772,Полуприцеп-цистерна 96226-0000010-04год изгото...,NaN,NaN,NaN,NaN,NaN,GAZ,NaN,Полуприцеп-цистерна 96226-0000010-04год изгото...,None,None
7498,29587,2014-000-001202-627/1,NaN,прицеп-ёмкость ХТС 10гос.рег.знак НМ9853 37198...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,прицеп-ёмкость ХТС 10гос.рег.знак НМ9853 37198...,None,None
7499,29588,2014-000-001202-627/1,NaN,прицеп 2ПТС-4мод. 887Агос.рег.знак НМ9873 3719...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,прицеп 2ПТС-4мод. 887Агос.рег.знак НМ9873 3719...,None,None
7500,29589,2014-000-001202-627/1,NaN,прицеп 2ПТС-4мод.887Бгос.рег.знак НМ9869 37199...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,прицеп 2ПТС-4мод.887Бгос.рег.знак НМ9869 37199...,None,None
7501,29590,2014-000-001202-627/1,NaN,прицеп 2ПТС-4Сгос.рег.знак НМ9871 371993 г.в.к...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,прицеп 2ПТС-4Сгос.рег.знак НМ9871 371993 г.в.к...,None,None
7502,29591,2014-000-001202-627/1,NaN,прицеп 2ПТС-4мод.887гос.рег.знак НМ9872 371986...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,прицеп 2ПТС-4мод.887гос.рег.знак НМ9872 371986...,None,None
7503,29592,2014-000-001202-627/1,NaN,прицеп 2ПТС-4мод.887Бгос.рег.знак НМ 9874 3719...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,прицеп 2ПТС-4мод.887Бгос.рег.знак НМ 9874 3719...,None,None
7504,29593,2014-000-001202-627/1,NaN,прицеп 2ПТС-4мод. 9511гос.рег.знак НВ 2310 371...,NaN,NaN,NaN,NaN,GAZ 2310,NaN,NaN,прицеп 2ПТС-4мод. 9511гос.рег.знак НВ 2310 371...,None,2310
7505,29594,2014-000-001202-627/1,NaN,прицеп 2ПТСЕ-45гос.рег.знак НВ 5547 371995 г.в...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,прицеп 2ПТСЕ-45гос.рег.знак НВ 5547 371995 г.в...,None,None


In [81]:
data[data['Brand_NER'] == 'hyundai'][data['Model_NER'].isnull()]

/var/folders/2t/mb7xtxlx289fhmxl983vql2c0000gp/T/ipykernel_72270/819527879.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data[data['Brand_NER'] == 'hyundai'][data['Model_NER'].isnull()]


,id,notificationreferencenumber,vehicleproperty_vin,vehicleproperty_description_short,nBody,nChas,BrandModel,BrandModel2,BrandModel3,Brand,Brand2,with_space,Brand_NER,Model_NER
1126,3726,2014-000-001249-286,XU527666A80000641,HYUNDAI 27666А Грузовой-фургон 2008 г. в.,NaN,NaN,NaN,NaN,HYUNDAI PORTER/H-100,BAW,NaN,HYUNDAI 27666А Грузовой-фургон 2008 г. в.,hyundai,None
1764,6111,2014-000-001531-406,NaN,экскаватор HYUNDAI R17OW-7год выпуска 2011заво...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,экскаватор HYUNDAI R17OW-7год выпуска 2011за...,hyundai,None
2378,7639,2014-000-002236-622,NaN,ПОГРУЗЧИК HYUNDAI HSL850-7Aг/н 86 УЕ 9501 ПТС ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ПОГРУЗЧИК HYUNDAI HSL850-7Aг/н 86 УЕ 9501 ПТ...,hyundai,None
3968,16754,2014-000-003745-273,XWELB41CBE0002820,HyundaiHyundai i402014Серый,NaN,NaN,NaN,NaN,NaN,GENESIS,NaN,Hyundai Hyundai i402014Серый,hyundai,None
4375,18672,2014-000-003988-001,X894740PT80AB9407,Модель автомобиля: отсутствует. 4740PT (HYUNDA...,NaN,NaN,NaN,NaN,NaN,5TSARZ,NaN,Модель автомобиля: отсутствует. 4740PT ( HYUND...,hyundai,None
4732,19600,2014-000-004345-279,KMJNN19RP5C301026,ПТС: 25 УА 532163 от 23.12.2009 Автобус HYUNDA...,NaN,NaN,NaN,NaN,NaN,HYUNDAI,NaN,ПТС: 25 УА 532163 от 23.12.2009 Автобус HYUND...,hyundai,None
5057,20575,2014-000-004630-774,KMHWH81JACU437021,HYUNDAI H-1 2.5 MT 2011г.,NaN,NaN,HYUNDAI H-1,NaN,NaN,HYUNDAI,NaN,HYUNDAI H-1 2.5 MT 2011г.,hyundai,None
5459,21919,2014-000-005028-809,X893797PM90AB9511,Модель автомобиля: отсутствует. 3797 PM (HYUND...,NaN,NaN,NaN,NaN,NaN,5TSARZ,NaN,Модель автомобиля: отсутствует. 3797 PM ( HYUN...,hyundai,None
5726,22787,2014-000-005282-967,KMHWH81RBBU371372,Автомобиль легковой HUNDAI H-1 2.4ATгод изгото...,NaN,NaN,NaN,NaN,NaN,HYUNDAI,NaN,Автомобиль легковой HUNDAI H-1 2.4ATгод из...,hyundai,None
6292,24915,2014-000-004196-616,HHKHZ412JC0000078,Экскаватор HYUNDAI R140W-9S2012 г. Цвет: желты...,NaN,NaN,NaN,NaN,KAMAZ 5410,NaN,NaN,Экскаватор HYUNDAI R140W-9S 2012 г. Цвет: ж...,hyundai,None


In [ ]:
df3 = df2[df2['Brand_NER'] == 'ford']

In [86]:
models = jsonlines.open('data/brands/FORD.jsonl')
rules_model = create_rules(models, label='MODEL')
# print(rules_model)
with jsonlines.open('data/brands/FORD_rules.jsonl', mode='w') as writer:
    writer.write_all(rules_model)

nlp = spacy.load("ru_core_news_sm")

config = { "overwrite_ents": True }
ruler = nlp.add_pipe("entity_ruler", before="ner", config=config).from_disk("data/brands/FORD_rules.jsonl", )

ents = []
for article in tqdm(df3['with_space'].values):
    article = str(article)
    doc = nlp(article)
    list_brands = [ent.ent_id_ for ent in doc.ents if ent.label_ == 'MODEL']
    ents.append(None if len(list_brands) == 0 else list_brands[0])

df3 = df3.assign(Model_NER=ents)

100%|████████████████████████| 305/305 [00:04<00:00, 71.72it/s]


In [90]:
df3[df3['Model_NER'].isnull()]

,id,notificationreferencenumber,vehicleproperty_vin,vehicleproperty_description_short,nBody,nChas,BrandModel,BrandModel2,BrandModel3,Brand,Brand2,with_space,Brand_NER,Model_NER
2201,7365,2014-000-002056-357,X9F5XXEED57K63424,FORD FOKUS2007,NaN,NaN,NaN,NaN,NaN,FORD,NaN,FORD FOKUS2007,ford,None
2998,9569,2014-000-002871-481,1FTRW14W95FA83085,Модель автомобиля: FORD F-150год выпуска:2005,NaN,NaN,NaN,NaN,NaN,FORD,NaN,Модель автомобиля: FORD F-150год выпуска:2005,ford,None
3086,9770,2014-000-002980-660,WF0MXXGBWM7D60094,Модель автомобиля: FORD GALAXYгод выпуска:2007,NaN,NaN,NaN,NaN,NaN,FORD,NaN,Модель автомобиля: FORD GALA XYгод выпуска:...,ford,None
3136,10003,2014-000-003052-800,WJMS2NSK00C105550,Грузовой тягач седельный IVICO Stralis FORD AS...,NaN,NaN,NaN,NaN,NaN,FORD,NaN,Грузовой тягач седельный IVICO Stralis FORD ...,ford,None
6113,23757,2014-000-005578-095,1FTFW1EV6AFB24193,Модель автомобиля: FORD F150год выпуска:2009,NaN,NaN,NaN,FORD CARGO,FORD CARGO,FORD,NaN,Модель автомобиля: FORD F150год выпуска:2009,ford,None
